In [113]:
using LinearAlgebra;
using SpecialFunctions;
using ControlSystems;
using Plots;
using VectorizedRoutines.Matlab

## LegT Measure


In [114]:
function legt_ss(dim::Integer, theta::Real)
    B = (2 * [0:dim-1;] .+ 1) .^ (1 / 2)
    A = -inv(theta) .* (B * B') .* Matrix{Integer}((-1) .^ triu([0:dim-1;] * ones(dim)' - ones(dim) * [0:dim-1;]'))

    legt = ss(A, B, Matrix(I, dim, dim), 0)
    return legt
end

legt_ss (generic function with 1 method)

In [115]:
# N=10
# k, n = meshgrid(0:(N-1), 0:(N-1))
# k
# n

In [116]:
# N=10
# k, n = meshgrid([0:N-1;], [0:N-1;])
# k
# n

In [117]:
function build_LegT(N::Integer, λ_n::Integer=1)
    k, n = meshgrid([0:N-1;], [0:N-1;])
    case = -1 ^ (n - k)
    A = missing
    B = missing

    if λ_n == 1
        A_base = sqrt((2 .* n) .+ 1) .* sqrt((2 .* k) .+ 1)
        pre_D = sqrt.(Diagonal((2 .* [0:N-1;]) .+ 1))
        B = D = dropdims(Diagonal(pre_D), dims=1)
        A = @.ifelse(
            k <= n,
             A_base, 
             A_base .* case
            )  # if n >= k, then case_2 * A_base is used, otherwise A_base

    elseif λ_n == 2
        A_base = (-1) .* ((2 .* n) .+ 1) 
        B = Diagonal(((2 .* [0:N-1;]) .+ 1) .* ((-1) .^ (n)))[:, None]
        A = @.ifelse(
            k <= n, 
            A_base * case, 
            A_base
            )  # if n >= k, then case_2 * A_base is used, otherwise A_base
    else
        error("lambda must be 1 or 2")

    end

    return A, B
end

build_LegT (generic function with 2 methods)

## LagT Measure


In [118]:
function lagt_ss(dim::Integer, alpha::Real=0, beta::Real=1)
    A = -tril(ones(dim, dim), -1) - ((1 + beta) / 2) * I
    B = Vector{Float64}(1:dim)
    B = gamma.((B .+ alpha)) ./ (gamma.(B) .* gamma(alpha + 1))

    L = exp.(0.5 .* (loggamma.((B .+ beta)) .- loggamma.(B)))
    A = inv.(L) .* A .* L
    B = inv.(L) .* B .* exp.(-0.5 .* loggamma.((B .+ beta))) .* beta^((1 - alpha) / 2)

    lagt = ss(A, B, Matrix(I, dim, dim), 0)
    return lagt
end

function lagt_ss(dim::Integer, beta::Real=1)
    return lagt_ss(dim, 0, beta)
end

function lagt_ss(dim::Integer)
    return lagt_ss(dim, 1)
end

lagt_ss (generic function with 3 methods)

In [119]:
function build_LagT(N::Integer, alpha::Real=0, beta::Real=1)
    A = missing
    B = missing

    q = [0:N-1]

    L = exp.(0.5 .* (loggamma.((q .+ alpha .+ 1)) .- loggamma.(q .+ 1)))
    inv_L = 1.0 ./ (L[:, None])

    pre_A = (Matrix{Float64}(I, N, N) .* ((1 .+ beta) ./ 2)) .+ UnitLowerTriangular((ones(N, N) .* Matrix(0.0I, N, N)))
    pre_B = (binomial((alpha .+ q), q))[:, None]
    A = -inv_L .* pre_A .* L[None, :]
    B = (
        exp(-0.5 .* loggamma.(1 .- alpha))
        .* (beta .^ ((1 .- alpha) ./ 2))
        .* inv_L
        .* pre_B
    )

    return A, B
end

build_LagT (generic function with 3 methods)

## LegS Measure

In [120]:
function legs_ss(dim::Integer)
    q = [0:dim-1;]
    r = 2 * q .+ 1
    M = -(tril(ones(dim) * r') - diagm(q))
    T = sqrt(diagm(r))
    A = T * M * inv(T)
    B = diag(T)

    legs = ss(A, B, Matrix(I, dim, dim), 0)
    return legs
end

legs_ss (generic function with 1 method)

In [121]:
function build_LegS(N::Integer)
    k, n = meshgrid(0:(N-1), 0:(N-1))
    pre_D = sqrt(Diagonal(2 .* q .+ 1))
    B = D = Diagonal(pre_D)[:, None]

    A_base = (-sqrt(2 .* n .+ 1)) .* sqrt(2 .* k .+ 1)
    case_2 = (n .+ 1) ./ (2 .* n .+ 1)

    A = @.ifelse(
        n > k, A_base, 0.0
        ) # if n > k, then A_base is used, otherwise 0
    A = @.ifelse(
        n == k, (A_base .* case_2), A
    )  # if n == k, then A_base is used, otherwise A

    return A, B
end

build_LegS (generic function with 1 method)

## Make State-Space System

In [122]:
function cont_or_disc(A, B, C, D, step::Integer=nothing)
    sys = nothing
    
    if step == nothing
        sys = ss(A, B, C, D) # continuous-time system
    else
        sys = ss(A, B, C, D, step) # discrete-time system
    end

    return sys
end

cont_or_disc (generic function with 2 methods)

## Visualizations 

#### Bryan's

## LegT

**LegT Discrete**

In [123]:
anim = @animate for i = 1:60
    step = 1
    A, B =  build_LegT(i, 1);
    C, D = ones(i, i), 0
    println("A = ", A)
    println("\nB = ", B)
    println("\nC = ", C)
    println("\nD = ", D)
    legt = cont_or_disc(A, B, C, D, step);
    y, t, x = impulse(legt, 1.5)
    plot(t,x', xlabel="Time", xlims=(0, 1.5), ylimit=(-3, 3), title=("HiPPO-LegT Unit Impulse Response\nθ=1, n="*string(i)), lab=nothing)
end

gif(anim, "Bryan's Discrete HiPPO-LegT_Unit_Impulse_Response.gif", fps = 15);
legt = legt_ss(15,1)
legt = build_LegT(15, 1)

ArgumentError: ArgumentError: dropped dims must all be size 1

**LegT Continuous**

In [124]:
anim = @animate for i = 1:60
    A, B = build_LegT(i, 1);
    C, D = Matrix(I, i, i), 0
    legt = cont_or_disc(A, B, C, D);
    y, t, x = impulse(legt, 1.5)
    plot(t,x', xlabel="Time", xlims=(0, 1.5), ylimit=(-3, 3), title=("HiPPO-LegT Unit Impulse Response\nθ=1, n="*string(i)), lab=nothing)
end

gif(anim, "Bryan's Continuous HiPPO-LegT_Unit_Impulse_Response.gif", fps = 15);
legt = legt_ss(15,1)
legt = build_LegT(15, 1)

A = [1.0;;]

B = [1.0]

C = [1.0;;]

D = 0


MethodError: MethodError: no method matching cont_or_disc(::Matrix{Float64}, ::Base.ReshapedArray{Float64, 1, Diagonal{Float64, Vector{Float64}}, Tuple{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64}}}, ::Matrix{Bool}, ::Int64, ::Nothing)
Closest candidates are:
  cont_or_disc(::Any, ::Any, ::Any, ::Any) at ~/Documents/Coding/s4former/notebooks/notebooks-jl/HiPPO-Julia.ipynb:1
  cont_or_disc(::Any, ::Any, ::Any, ::Any, !Matched::Integer) at ~/Documents/Coding/s4former/notebooks/notebooks-jl/HiPPO-Julia.ipynb:1

## LegS

**LegS Discrete**

In [125]:
anim = @animate for i = 1:60
    step = 0.001
    A, B = build_LegS(i);
    C, D = Matrix(I, i, i), 0
    legt = cont_or_disc(A, B, C, D, step);
    y, t, x = impulse(legt, 1.5)
    plot(t,x', xlabel="Time", xlims=(0, 1.5), ylimit=(-3, 3), title=("HiPPO-LegT Unit Impulse Response\nθ=1, n="*string(i)), lab=nothing)
end

gif(anim, "Bryan's Discrete HiPPO-LegT_Unit_Impulse_Response.gif", fps = 15);
legs = legs_ss(15)
legs = build_LegS(15)

UndefVarError: UndefVarError: q not defined

**LegS Continuous**

In [126]:
anim = @animate for i = 1:60
    A, B = build_LegS(i);
    C, D = Matrix(I, i, i), 0
    legt = cont_or_disc(A, B, C, D);
    y, t, x = impulse(legt, 1.5)
    plot(t,x', xlabel="Time", xlims=(0, 1.5), ylimit=(-3, 3), title=("HiPPO-LegT Unit Impulse Response\nθ=1, n="*string(i)), lab=nothing)
end

gif(anim, "Bryan's Continuous HiPPO-LegT_Unit_Impulse_Response.gif", fps = 15);
legs = legs_ss(15)
legs = build_LegS(15)

UndefVarError: UndefVarError: q not defined

## LagT

**LagT Discrete**

In [127]:
anim = @animate for i = 1:60
    step = 0.001
    A, B = build_LagT(i, 0, 1);
    C, D = Matrix(I, i, i), 0
    legt = cont_or_disc(A, B, C, D, step);
    y, t, x = impulse(legt, 1.5)
    plot(t,x', xlabel="Time", xlims=(0, 1.5), ylimit=(-3, 3), title=("HiPPO-LegT Unit Impulse Response\nθ=1, n="*string(i)), lab=nothing)
end

gif(anim, "Bryan's Discrete HiPPO-LegT_Unit_Impulse_Response.gif", fps = 15);
lagt = lagt_ss(15,1)
lagt = build_LagT(15, 0, 1)

MethodError: MethodError: no method matching +(::UnitRange{Int64}, ::Int64)
For element-wise addition, use broadcasting with dot syntax: array .+ scalar
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at /opt/julia-1.7.3/share/julia/base/operators.jl:655
  +(!Matched::T, ::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} at /opt/julia-1.7.3/share/julia/base/int.jl:87
  +(!Matched::Union{VectorizationBase.AbstractMask{W}, VectorizationBase.VecUnroll{<:Any, W, SIMDTypes.Bit, <:VectorizationBase.AbstractMask{W}}} where W, ::Union{Bool, Float16, Float32, Float64, Int16, Int32, Int64, Int8, UInt16, UInt32, UInt64, UInt8, SIMDTypes.Bit}) at ~/.julia/packages/VectorizationBase/MvgPF/src/base_defs.jl:348
  ...

**LagT Continuous**

In [128]:
anim = @animate for i = 1:60
    A, B = build_LagT(i, 0, 1);
    C, D = Matrix(I, i, i), 0
    legt = cont_or_disc(A, B, C, D);
    y, t, x = impulse(legt, 1.5)
    plot(t,x', xlabel="Time", xlims=(0, 1.5), ylimit=(-3, 3), title=("HiPPO-LegT Unit Impulse Response\nθ=1, n="*string(i)), lab=nothing)
end

gif(anim, "Bryan's Continuous HiPPO-LegT_Unit_Impulse_Response.gif", fps = 15);
lagt = lagt_ss(15,1)
lagt = build_LagT(15, 0, 1)

MethodError: MethodError: no method matching +(::UnitRange{Int64}, ::Int64)
For element-wise addition, use broadcasting with dot syntax: array .+ scalar
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at /opt/julia-1.7.3/share/julia/base/operators.jl:655
  +(!Matched::T, ::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} at /opt/julia-1.7.3/share/julia/base/int.jl:87
  +(!Matched::Union{VectorizationBase.AbstractMask{W}, VectorizationBase.VecUnroll{<:Any, W, SIMDTypes.Bit, <:VectorizationBase.AbstractMask{W}}} where W, ::Union{Bool, Float16, Float32, Float64, Int16, Int32, Int64, Int8, UInt16, UInt32, UInt64, UInt8, SIMDTypes.Bit}) at ~/.julia/packages/VectorizationBase/MvgPF/src/base_defs.jl:348
  ...

#### Eric's

**LegT**

In [129]:
anim = @animate for i = 1:60
    legt = legt_ss(i,1);
    y, t, x = impulse(legt, 1.5)
    plot(t,x', xlabel="Time", xlims=(0, 1.5), ylimit=(-3, 3), title=("HiPPO-LegT Unit Impulse Response\nθ=1, n="*string(i)), lab=nothing)
end

gif(anim, "HiPPO-LegT_Unit_Impulse_Response.gif", fps = 15);
legt = legt_ss(15,1)

┌ Info: Saved animation to 
│   fn = /home/beegass/Documents/Coding/s4former/notebooks/notebooks-jl/HiPPO-LegT_Unit_Impulse_Response.gif
└ @ Plots /home/beegass/.julia/packages/Plots/E2187/src/animation.jl:126


StateSpace{Continuous, Float64}
A = 
 -1.0                  1.7320508075688772   -2.23606797749979     2.6457513110645907   -3.0                  3.3166247903554     -3.605551275463989    3.872983346207417    -4.123105625617661     4.358898943540674   -4.58257569495584     4.795831523312719   -5.0                  5.196152422706632   -5.385164807134504
 -1.7320508075688772  -2.9999999999999996    3.872983346207417   -4.58257569495584      5.196152422706632   -5.744562646538029    6.244997998398397   -6.7082039324993685    7.1414284285428495   -7.54983443527075     7.937253933193771   -8.306623862918073    8.660254037844386   -9.0                  9.327379053088814
 -2.23606797749979    -3.872983346207417    -5.000000000000001    5.916079783099617    -6.708203932499369    7.416198487095663   -8.06225774829855     8.660254037844387    -9.219544457292887     9.746794344808965  -10.246950765959598   10.723805294763608  -11.180339887498949   11.618950038622252  -12.041594578792296
 -2.64575

**LegS**

In [130]:
anim = @animate for i = 1:60
    legs = legs_ss(i);
    y, t, x = impulse(legs, 1.5)
    plot(t,x', xlabel="Time", xlims=(0, 1.5), ylimit=(-3, 3), title=("HiPPO-LegS Unit Impulse Response\nθ=1, n="*string(i)), lab=nothing)
end

gif(anim, "HiPPO-LegS_Unit_Impulse_Response.gif", fps = 15);
legs = legs_ss(15)

┌ Info: Saved animation to 
│   fn = /home/beegass/Documents/Coding/s4former/notebooks/notebooks-jl/HiPPO-LegS_Unit_Impulse_Response.gif
└ @ Plots /home/beegass/.julia/packages/Plots/E2187/src/animation.jl:126


StateSpace{Continuous, Float64}
A = 
 -1.0                  0.0                   0.0                   0.0                  0.0                  0.0                  0.0                  0.0                  0.0                  0.0                  0.0                  0.0                  0.0                 0.0                  0.0
 -1.7320508075688772  -2.0                   0.0                   0.0                  0.0                  0.0                  0.0                  0.0                  0.0                  0.0                  0.0                  0.0                  0.0                 0.0                  0.0
 -2.23606797749979    -3.8729833462074175   -3.0                   0.0                  0.0                  0.0                  0.0                  0.0                  0.0                  0.0                  0.0                  0.0                  0.0                 0.0                  0.0
 -2.6457513110645907  -4.582575694955841    -5.9160797830996

**LagT**

In [131]:
anim = @animate for i = 1:60
    lagt = lagt_ss(i, 0, 1);
    y, t, x = impulse(lagt, 1.5)
    plot(t,x', xlabel="Time", xlims=(0, 1.5), ylimit=(-3, 3), title=("HiPPO-LagT Unit Impulse Response\nθ=1, n="*string(i)), lab=nothing)
end

gif(anim, "HiPPO-LagT_Unit_Impulse_Response.gif", fps = 15);
lagt = lagt_ss(15, 0, 1)

┌ Info: Saved animation to 
│   fn = /home/beegass/Documents/Coding/s4former/notebooks/notebooks-jl/HiPPO-LagT_Unit_Impulse_Response.gif
└ @ Plots /home/beegass/.julia/packages/Plots/E2187/src/animation.jl:126


StateSpace{Continuous, Float64}
A = 
 -1.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -1.0  -1.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -1.0  -1.0  -1.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -1.0  -1.0  -1.0  -1.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1